In [1]:
#| default_exp popgen

In [2]:
#| export
# chewc/popgen.py
from typing import Tuple
import jax
import jax.numpy as jnp
from numpy.random import default_rng


import msprime
# Correctly import the Population structure from your new module
from chewc.structs import Population, SimParam, SimConfig


In [8]:
#| export
# In your refactored chewc/popgen.py
from typing import Tuple
from chewc.structs import Population # Import the NEW Population PyTree
import jax
import jax.numpy as jnp

def create_founders(
    key: jax.random.PRNGKey,
    n_ind: int,
    n_chr: int,
    n_loci_per_chr: int,
    ploidy: int = 2,
    chr_len_cm: float = 100.0
) -> Tuple[Population, jnp.ndarray]:
    """
    Creates a dense founder population and a corresponding genetic map.

    Returns:
        A tuple of (Population, genetic_map). The Population object is a lean
        PyTree with no padding, and the genetic_map is a JAX array.
    """
    key_geno, _ = jax.random.split(key)

    # 1. Generate dense genetic data
    founder_geno = jax.random.randint(
        key_geno, (n_ind, n_chr, ploidy, n_loci_per_chr), 0, 2, dtype=jnp.uint8
    )

    # 3. Create dense metadata
    ids = jnp.arange(n_ind, dtype=jnp.int32)
    mother_ids = jnp.full(n_ind, -1, dtype=jnp.int32)
    father_ids = jnp.full(n_ind, -1, dtype=jnp.int32)
    birth_gens = jnp.zeros(n_ind, dtype=jnp.int32)
    founder_meta = jnp.stack([ids, mother_ids, father_ids, birth_gens], axis=-1)

    # 4. Generate a uniform genetic map
    loci_pos = jnp.linspace(0., chr_len_cm, n_loci_per_chr)
    genetic_map = jnp.tile(loci_pos, (n_chr, 1))

    # 5. Instantiate the Population PyTree and return with the map
    population = Population(
        geno=founder_geno,
        meta=founder_meta
    )

    return population, genetic_map

In [9]:
#| hide
import nbdev; nbdev.nbdev_export()